In [ ]:
# pip install \
#   groq requests \
#   langgraph langchain langchain-community langchain-groq \
#   llama-index llama-index-llms-groq

In [19]:
import requests
import dotenv

dotenv.load_dotenv()

True

In [73]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

llm.invoke("hello")

AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 11, 'total_tokens': 35, 'completion_time': 0.055389591, 'completion_tokens_details': None, 'prompt_time': 4.7969e-05, 'prompt_tokens_details': None, 'queue_time': 0.049360271, 'total_time': 0.05543756}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_37da608fc1', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b7291-7dc0-75d0-b8bf-0e29fe68e2a5-0', usage_metadata={'input_tokens': 11, 'output_tokens': 24, 'total_tokens': 35})

In [70]:
import ast

def local_math(expression: str) -> str:
    """for calculating any mathematical expressions
        args: math expression as string
        return: string
    """
    try:
        # builtins rm access to open, exec, __import__, print
        # this restricts - eval("__import__('os').system('rm -rf /')")
        return str(eval(expression, {"__builtins__": {}}))
    except Exception as e:
        return f"Math error: {e}"

def guess_gender(name: str) -> str:
    """API to guess gender from a name
        arg: name as string
        return: string
    """
    r = requests.get("https://api.genderize.io", params={"name": name}, timeout=5)
    return str(r.json()['gender']) # json object - {"count":6440, "name":"ajay", "gender":"male", "probability":0.99}


In [71]:
local_math("3*7")

'21'

In [72]:
guess_gender("ajay")

'male'

In [53]:
from langchain.agents import create_agent


tools = [local_math, guess_gender]

prompt = (
    "You have access to a tools that can do math and guess gender based on name given. "
    "Use the tool to help answer user queries."
)
agent = create_agent(llm, tools, system_prompt=prompt)


messages = [
    {"role": "user", "content": "What is 4*2+2"},
    {"role": "user", "content": "Guess the gender of velu"},
]

for event in agent.stream(
    {"messages": messages},
    stream_mode="values",
):
    print(event["messages"][-1].pretty_print())

================================ Human Message =================================

Guess the gender of velu
None
================================== Ai Message ==================================
Tool Calls:
  guess_gender (hjmxp24zg)
 Call ID: hjmxp24zg
  Args:
    name: velu
None
================================= Tool Message =================================
Name: guess_gender

male
None
================================== Ai Message ==================================

The gender of Velu is male.
None


In [62]:
from langgraph.graph import StateGraph
from typing import TypedDict, Optional

class AgentState(TypedDict):
    query: str
    result: Optional[str]

def router(state: AgentState):
    q = state["query"].lower()

    if any(c.isdigit() for c in q):
        return "math"

    if "gender" in q or "name" in q:
        return "api"

    return "no_tool"


def math_node(state: AgentState):
    return {"result": local_math(state["query"])}

def api_node(state: AgentState):
    name = state["query"].split()[-1]
    return {"result": guess_gender(name)}

def no_tool_node(state: AgentState):
    return {"result": "No tool available for this query"}


graph = StateGraph(AgentState)

graph.add_node("router", lambda s: s)
graph.add_node("math", math_node)
graph.add_node("api", api_node)
graph.add_node("no_tool", no_tool_node)

graph.set_entry_point("router")

graph.add_conditional_edges(
    "router",
    router,
    {
        "math": "math",
        "api": "api",
        "no_tool": "no_tool",
    },
)

graph.set_finish_point("math")
graph.set_finish_point("api")
graph.set_finish_point("no_tool")

agent = graph.compile()


m1 = agent.invoke({"query": "12 * 9"})
m2 = agent.invoke({"query": "Guess gender of rohitha"})
m3 = agent.invoke({"query": "Explain ai agents"})

print(m1)
print(m2)
print(m3)

{'query': '12 * 9', 'result': '108'}
{'query': 'Guess gender of rohitha', 'result': 'male'}
{'query': 'Explain ai agents', 'result': 'No tool available for this query'}


In [63]:
mermaid = agent.get_graph().draw_mermaid()
print(mermaid)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	router(router)
	math(math)
	api(api)
	no_tool(no_tool)
	__end__([<p>__end__</p>]):::last
	__start__ --> router;
	router -.-> api;
	router -.-> math;
	router -.-> no_tool;
	api --> __end__;
	math --> __end__;
	no_tool --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [48]:
from llama_index.llms.groq import Groq
from llama_index.core.agent.workflow import ReActAgent
from llama_index.core.workflow import Context

llm = Groq(model="meta-llama/llama-4-scout-17b-16e-instruct", temperature=0)
agent = ReActAgent(tools=[local_math, guess_gender], llm=llm, streaming=False)

# context to store the convo history/session state
ctx = Context(agent)

# 1st qn
handler1 = agent.run("What is 20+(2*4)?", ctx=ctx)
response1 = await handler1

print(response1)
print(response1.tool_calls)

# 2nd qn (same ctx)
handler2 = agent.run("What is the gender of rahul?", ctx=ctx)
response2 = await handler2

print(response2)
print(response2.tool_calls)


28
[ToolCallResult(tool_name='local_math', tool_kwargs={'expression': '20+(2*4)'}, tool_id='7f360737-4a44-4940-97f6-71cf2cf29215', tool_output=ToolOutput(blocks=[TextBlock(block_type='text', text='28')], tool_name='local_math', raw_input={'args': (), 'kwargs': {'expression': '20+(2*4)'}}, raw_output='28', is_error=False), return_direct=False)]
male
[ToolCallResult(tool_name='guess_gender', tool_kwargs={'name': 'rahul'}, tool_id='c1042e11-01aa-4873-ba23-828b37d8fb06', tool_output=ToolOutput(blocks=[TextBlock(block_type='text', text='male')], tool_name='guess_gender', raw_input={'args': (), 'kwargs': {'name': 'rahul'}}, raw_output='male', is_error=False), return_direct=False)]
